In [1]:
import pandas as pd
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc 
from dash.dependencies import Input, Output
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objs as go

from lime.lime_tabular import LimeTabularExplainer
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier


import operator


In [2]:
df = pd.read_pickle(r"C:\Users\user\Downloads\P7.pkl")
df.head()

,AMT_INCOME_TOTAL,AMT_CREDIT,DAYS_EMPLOYED,DAYS_BIRTH,FLAG_DOCUMENT_8,AMT_GOODS_PRICE,AMT_ANNUITY,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,TARGET
SK_ID_CURR,,,,,,,,,,,
100023,11.407565,13.207607,-2038.0,-11348.0,0.0,13.026953,17563.5,1.0,1.0,1.0,0.0
100043,12.196022,13.371055,-768.0,-17199.0,0.0,13.224017,23157.0,1.0,1.0,1.0,0.0
100047,12.218495,13.992468,-1262.0,-17482.0,0.0,13.658857,35028.0,0.0,0.0,1.0,1.0
100049,11.813030,12.573742,-3597.0,-13384.0,0.0,12.382125,16258.5,1.0,1.0,1.0,1.0
100059,13.199324,13.422468,-6977.0,-18632.0,0.0,13.422468,34596.0,0.0,1.0,1.0,0.0


In [3]:
df.DAYS_BIRTH = df.DAYS_BIRTH.apply(lambda x: abs(int(x/365)))
df = df.rename(columns={"DAYS_BIRTH":"AGE"})
num_columns = df.select_dtypes(include=["float64"]).columns
transf = ['AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_INCOME_TOTAL']
for var in transf:
    df[var] = np.exp(df[var]).astype(int)

In [4]:
X = df.iloc[:,0:9]
y = df.iloc[:,10]
rf = RandomForestClassifier(max_depth=25, min_samples_leaf=1, n_estimators=150)
rf = rf.fit(X, y)

In [5]:
def probability(X):
    probas = rf.predict_proba(X)
    probas = [proba[0] for proba in probas]
    return probas

In [6]:
df["RF_PROBA"]=probability(X)
df["RF_PRED"]=rf.predict(X)
df['Solvable'] = df["RF_PROBA"]
df['Non Solvable']= 1-df["RF_PROBA"]

In [7]:
# Interprétabilité du modèle
lime_explainer = LimeTabularExplainer(X,
                             feature_names=X.columns,
                             discretize_continuous=False)

In [8]:
nbrs = NearestNeighbors(n_neighbors=20, algorithm='ball_tree').fit(X)

In [9]:
external_stylesheets = [dbc.themes.LUX]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    
    dcc.Tabs([
        # Premier onglet
        dcc.Tab(label='Solvabilité Client', children=[

            html.Div([
                html.H3("Id Client"),
                dcc.Dropdown(
                id='id-client',
                options=[{'label': i, 'value': i} for i in X.index],
                value=X.index[0]
                ),
            ], className="m-5"),
            html.Div([
             
                html.Div([
                    html.H3("Probabilité de Solvabilité"),
                    dcc.Graph(id='proba',
                              figure={},
                              style={"height": 500,
                                     "width": 500}
                             ),
                ]),
             
                html.Div([
                    html.H3("Importance des paramètres pour le client"), 
                    dcc.Graph(id='graph',
                              figure={},
                              style={"height":500,
                                     "width":800}
                             ),       
                ], className=''),        
            ], className="m-5"),
            
            html.Div([
                    html.H3("Profil du client")], className="mx-5"), 
      
            html.Div(id='table_client', 
        
             className='m-5'),
    
            
            html.Div([
                html.H3("Profils de clients similaires")], className="mx-5"),
            
            html.Div(id='table_clients', 
        
             className='m-5'),
                
        ]),
       
    # Deuxième onglet  
    dcc.Tab(label='Exploration des données', children=[
           html.Div([
                html.Div([
                    dcc.Dropdown(
                        id='xaxis-column',
                        options=[{'label': i, 'value': i} for i in num_columns],
                        value='AMT_CREDIT'
                    ),
                    dcc.RadioItems(
                        id='xaxis-type',
                        options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                        value='Linear',
                        labelStyle={'display': 'inline-block'}
                    )
                ],
                style={'width': '48%', 'display': 'inline-block'}),

                html.Div([
                    dcc.Dropdown(
                        id='yaxis-column',
                        options=[{'label': i, 'value': i} for i in num_columns],
                        value='AMT_ANNUITY'
                    ),
                    dcc.RadioItems(
                        id='yaxis-type',
                        options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                        value='Linear',
                        labelStyle={'display': 'inline-block'}
                    )
                ],style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
            ]),

            dcc.Graph(id='indicator-graphic'),

        ]),
         ]),
     ])



@app.callback(
    Output('table_client', 'children'),
    [
    Input('id-client', 'value')])
def update_table(id_client):
    dff = X[X.index == id_client]

    return [dash_table.DataTable(
        id="table",
        columns=[
                       {"name": i, "id": i} for i in X.columns
                    ],
        data=dff.to_dict('records'))]
       
    
@app.callback(
    Output('table_clients', 'children'),
    [
     Input('id-client', "value")])
def update_table2(id_client):
    
    indices_similary_clients = nbrs.kneighbors(
        np.array(X.loc[id_client]).reshape(1, -1))[1].flatten()
    
    dff = X[X.index.isin(X.index[indices_similary_clients])]

    return [dash_table.DataTable(
        id="table2",
        columns=[
                       {"name": i, "id": i} for i in X.columns
                    ],
        data=dff.to_dict('records'))]


@app.callback(
    Output('proba', 'figure'),
    [Input('id-client', 'value')])
def proba_pie(id_client):
    values = df.loc[id_client]
    values = (values['Solvable'],values['Non Solvable'])

    return {
        'data': [go.Pie(labels=['Solvable', "Non Solvable"],
                        values=values,
                        marker_colors=["#2ecc71", "#e74c3c"],
                        hole=.5
                       )],
        'layout': go.Layout(margin=dict(b=100)
                           )
    }
    del values
    
@app.callback(
   Output('graph', 'figure'),
   [Input('id-client', 'value'),
   ])
def update_graphic(id_client):
    exp = lime_explainer.explain_instance(X.loc[id_client].values,
                                rf.predict_proba,
                                num_features=10)
    
    indices, values = [], []
    
    for ind, val in sorted(exp.as_list(), key=operator.itemgetter(1)):
        indices.append(ind)
        values.append(val)
    dat = pd.DataFrame(values, columns=["values"], index=indices)
    dat["positive"] = dat["values"]>0
    del indices, values
    
    return {
            
        'data': [go.Bar(
                    x=dat["values"],
                    y=dat.index,
                    orientation='h',
                    marker_color=list(dat.positive.map({True: '#e74c3c', False: '#2ecc71'}).values)
        )],
            
        'layout': go.Layout(
                            margin=dict(l=300, r=0, t=30, b=100)
                            )
    } 


@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('xaxis-column', 'value'),
     Input('yaxis-column', 'value'),
     Input('xaxis-type', 'value'),
     Input('yaxis-type', 'value')])
def update_graph_2(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type):
       
    traces = []
    solvable_labels = ["Solvable", "Non Solvable"]
    for i, target in enumerate(df.RF_PRED.unique()):
        filtered_df = df[df['RF_PRED'] == target].reset_index()
        traces.append(dict(
            x=filtered_df[xaxis_column_name],
            y=filtered_df[yaxis_column_name],
            text=filtered_df.index,
            mode='markers',
            opacity=0.7,
            marker={
                'color':list(filtered_df["RF_PRED"].map({0.0: '#e74c3c', 1.0: "#2ecc71"}).values),
                'size': 5,
                'line': {'width': 0.15, 'color': 'white'}
            },
            name=solvable_labels[i]
        ))   
        
    return {
        'data': traces,
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 40, 't': 10, 'r': 0},
            hovermode='closest'
        )
    }
            
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_0_0m1635426179.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash/deps/react@16.v2_0_0m1635426179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_0_0m1635426179.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_0_0m1635426179.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_0_0m1635426181.min.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_0_0m1635426179.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2021 11:06:26